In [ ]:
import h5py 
import dxchange
import sys

sys.path.insert(0, '..')
from utils import *
from rec import Rec

detector_pixelsize = 3.03751e-6
focusToDetectorDistance = 1.28  # [m]
# adjustments for the cone beam
sx0 = 1.286*1e-3
z10 = 5.5*1e-3#/entry_0000: AtomiumS2 - AtomiumS2_codedht_random_007nm/TOMO/sx0 1.286 mm
z1 = z10-sx0
z2 = focusToDetectorDistance - z1
distance = (z1 * z2) / focusToDetectorDistance
magnification = focusToDetectorDistance / z1
voxelsize = float(cp.abs(detector_pixelsize / magnification))
print(voxelsize)

In [ ]:

# path = '/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random_010nm/'
# path_out = '/data/vnikitin/ESRF/ID16A/20240924_h5/Chip/Chip_codedht_random_010nm/'

# def find_angle(filename):    
#     with open(filename, 'r',encoding="latin-1") as file:
#         for line in file:
#             if "motor_pos" in line:
#                 print(line.split())
#                 return float(line.split()[2])    
# with  h5py.File(f'{path_out}/data.h5','a') as fid:
    
#     for id in range(1800):        
#         fname = f'{path}Chip_codedht_random_010nm{id:04}.edf'
        
#         ang = find_angle(fname)        
#         fid['exchange/theta'][id]=ang    
#         print(id,ang)
#         # pshifts = find_pos_shifts(fname)
#         # print(fname)
#         # print(ang)  
#         # print(pshifts)  
#         # data[id] = dxchange.read_edf(fname)[0]
#         # theta[id] = ang
#         # pos_shifts[id] = pshifts

In [ ]:

def find_angle(filename):    
    with open(filename, 'r',encoding="latin-1") as file:
        for line in file:
            if "motor_pos" in line:
                print(line.split())
                return float(line.split()[2])        

def find_pos_shifts(filename):    
    with open(filename, 'r',encoding="latin-1") as file:
        for line in file:
            if "motor_pos" in line:
                print(line.split())
                return np.array([float(line.split()[5]),float(line.split()[6])])                

path = '/data/vnikitin/ESRF/ID16A/20240924/Chip/Chip_codedht_random_010nm/'
path_out = '/data/vnikitin/ESRF/ID16A/20240924_h5/Chip/Chip_codedht_random_010nm/'


with  h5py.File(f'{path_out}/data.h5','w') as fid:
    data = fid.create_dataset('/exchange/data',shape=(1800,2048,2048),dtype='uint16')
    data_white0 = fid.create_dataset('/exchange/data_white0',shape=(20,2048,2048),dtype='uint16')
    data_white1 = fid.create_dataset('/exchange/data_white1',shape=(20,2048,2048),dtype='uint16')
    data_dark = fid.create_dataset('/exchange/data_dark',shape=(20,2048,2048),dtype='uint16')
    theta = fid.create_dataset('/exchange/theta',shape=(1800),dtype='float32')
    shifts = fid.create_dataset('/exchange/shifts',shape=(1800,2),dtype='float32')
    shifts[:] = np.loadtxt(f'{path}correct.txt')[:1800]    
    attrs = fid.create_dataset('/exchange/attrs',shape=(1800,3),dtype='float32')
    attrs[:] = np.loadtxt(f'{path}attributes.txt')[:1800]    
    pos_shifts = fid.create_dataset('/exchange/pos_shifts',shape=(1800,2),dtype='float32')
    shifts[:] = np.loadtxt(f'{path}correct.txt')[:1800]    
    dvoxelsize = fid.create_dataset('/exchange/voxelsize',shape=(1,),dtype='float32')
    dvoxelsize[0]=voxelsize
    dsx0 = fid.create_dataset('/exchange/sx0',shape=(1,),dtype='float32')
    dsx0[0] = sx0
    dz1 = fid.create_dataset('/exchange/z1',shape=(1,),dtype='float32')
    dz1[0]=z10
    ddetector_pixelsize = fid.create_dataset('/exchange/detector_pixelsize',shape=(1,),dtype='float32')
    ddetector_pixelsize[0]=detector_pixelsize
    dfocusdetectordistance = fid.create_dataset('/exchange/focusdetectordistance',shape=(1,),dtype='float32')
    dfocusdetectordistance[0]=focusToDetectorDistance



    for id in range(data_white0.shape[0]):
        fname = f'{path}ref{id:04}_0000.edf'
        data_white0[id] = dxchange.read_edf(fname)[0]

    for id in range(data_white1.shape[0]):
        fname = f'{path}ref{id:04}_1800.edf'
        data_white1[id] = dxchange.read_edf(fname)[0]

    for id in range(data_dark.shape[0]):
        fname = f'{path}darkend{id:04}.edf'
        data_dark[id] = dxchange.read_edf(fname)[0]

    for id in range(data.shape[0]):        
        fname = f'{path}Chip_codedht_random_010nm{id:04}.edf'
        
        ang = find_angle(fname)        
        
        pshifts = find_pos_shifts(fname)
        print(fname)
        print(ang)  
        print(pshifts)  
        data[id] = dxchange.read_edf(fname)[0]
        theta[id] = ang
        pos_shifts[id] = pshifts

